In [5]:
import os
import os.path as osp
from sys import argv
import time

import mmcv
import wandb
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from torch.utils.data import DataLoader

In [17]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1805109 (thesis_bank_document). Use `wandb login --relogin` to force relogin


True

In [47]:
def train():
    cfg = Config.fromfile(f"/media/quadro/NVME/Mehrab/Current_Experiment/config.py")

    cfg.gpu_ids = [2,3]

    set_random_seed(0, deterministic=False)

        
    val = True
    # Build dataset

    datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)]


    # Build the detector
    model = build_detector(cfg.model)
    # Add an attribute for visualization convenience
    model.CLASSES = datasets[0].CLASSES

    # Create work_dir
    mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
    train_detector(model, datasets, cfg, distributed=True, validate=val)

In [48]:
import torch
from torch.distributed.launch import launch

ImportError: cannot import name 'launch' from 'torch.distributed.launch' (/media/quadro/NVME/afnan/conda/openmmlab/lib/python3.8/site-packages/torch/distributed/launch.py)

In [34]:
# Load annotations before creating DataLoader
for dataset in datasets:
    dataset.load_annotations()

# Modify the DataLoader for distributed training
for i, dataset in enumerate(datasets):
    datasets[i] = DataLoader(
        dataset,
        batch_size=cfg.data.samples_per_gpu,
        shuffle=True,
        num_workers=cfg.data.workers_per_gpu,
        pin_memory=False,
        drop_last=True,
        sampler=DistributedSampler(dataset),
    )

TypeError: init_dist() missing 1 required positional argument: 'launcher'

In [7]:
import os
import os.path as osp
from sys import argv
import time
import warnings

import wandb
import mmcv
import torch
import torch.distributed as dist
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.utils import get_git_hash

from mmdet import __version__
from mmdet.apis import init_random_seed, set_random_seed, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.utils import (collect_env, get_device, get_root_logger,
                         replace_cfg_vals, setup_multi_processes,
                         update_data_root)

In [11]:
cfg = Config.fromfile(f"/media/quadro/NVME/Mehrab/Current_Experiment/config.py")

In [53]:
cfg = replace_cfg_vals(cfg)
update_data_root(cfg)

In [54]:
setup_multi_processes(cfg)

/media/quadro/NVME/Mehrab/Current_Experiment/mmdet/utils/setup_env.py:17: UserWarning: Multi-processing start method `fork` is different from the previous setting `spawn`.It will be force set to `fork`. You can change this behavior by changing `mp_start_method` in your config.
  warnings.warn(
/media/quadro/NVME/Mehrab/Current_Experiment/mmdet/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/media/quadro/NVME/Mehrab/Current_Experiment/mmdet/utils/setup_env.py:48: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(


In [55]:
cfg.gpu_ids = [2,3]

In [56]:
# create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

In [57]:
model = build_detector(
        cfg.model,
        train_cfg=cfg.get('train_cfg'),
        test_cfg=cfg.get('test_cfg'))
model.init_weights()

2024-01-21 21:37:38,948 - mmcv - INFO - initialize DLANetMMDet3D with init_cfg {'type': 'Pretrained', 'checkpoint': 'http://dl.yf.io/dla/models/imagenet/dla34%2Btricks-24a49e58.pth'}
2024-01-21 21:37:38,949 - mmcv - INFO - load model from: http://dl.yf.io/dla/models/imagenet/dla34%2Btricks-24a49e58.pth
2024-01-21 21:37:38,951 - mmcv - INFO - load checkpoint from http path: http://dl.yf.io/dla/models/imagenet/dla34%2Btricks-24a49e58.pth
Downloading: "http://dl.yf.io/dla/models/imagenet/dla34%2Btricks-24a49e58.pth" to /home/quadro/.cache/torch/hub/checkpoints/dla34%2Btricks-24a49e58.pth
 45%|████▍     | 27.0M/60.3M [03:16<04:01, 144kB/s] 


KeyboardInterrupt: 

In [ ]:
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)]

In [ ]:
model.CLASSES = datasets[0].CLASSES

In [8]:
meta = dict()
    # log env info
env_info_dict = collect_env()
env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])

In [9]:
timestamp =  timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())

In [12]:
meta['env_info'] = env_info
meta['config'] = cfg.pretty_text

In [15]:
seed = set_random_seed(0, deterministic=False)
meta['seed'] = seed
meta['exp_name'] = osp.basename("multi_gpu_test")


In [ ]:
train_detector(
        model,
        datasets,
        cfg,
        distributed=True,
        validate=True,
        timestamp=timestamp,
        meta=meta
    )

In [17]:
!bash torch ${@:3}

bash: torch: No such file or directory
